In [202]:
"""Lennard-Jones for specific atom"""

import numpy as np

class atom_props():
    def __init__(self, sigma, epsilon):
        self.sig = sigma
        self.eps = epsilon

def pot_LJ(atom,r):
    sig = atom.sig
    eps = atom.eps
    return 4*eps*((sig/r)**12 - (sig/r)**6)   

def pot_LJ_dl(r):
    u = np.zeros(r.shape)
    u[r != 0] = 4*((1/r[r != 0])**12 - (1/r[r != 0])**6)   
    u[r == 0] = 0 
    return u

def grad_pot_LJ_dl(r):
    u = np.zeros(r.shape)
    u[r != 0] = -24*(2*(1/r[r != 0])**13 - (1/r[r != 0])**7)   
    u[r == 0] = 0 
    return u

# Initiazlize box and particles
def particle_generator(L, N, dim):
    x = np.random.random((dim,N))*L
    v = np.zeros((dim,N),dtype=float)
    return (x,v)

def particle_dist(L, N, x, dim):
    # Particles NN, differences, coordinates

    # Calcolate distances to NN
    r = np.zeros((N,N),dtype=float)
    NN_max_dist = L/2
    xv = np.zeros((dim,N,N),dtype=float)
    delta_tot = np.zeros((dim,N,N))
    
    for i in range(dim):
        # Difference between coordinates per dimension
        delta = x[i,:].reshape(1,N)-x[i,:].reshape(N,1)

        # look for NN, define 'virtual' points
        xv[i,:,:] =  np.tile(x[i,:].reshape(N,1),N)
        xv[i][delta > NN_max_dist] = L + xv[i][delta > NN_max_dist] 
        xv[i][delta < -NN_max_dist] = -L + xv[i][delta < -NN_max_dist] 

        # New difference including 'virtual' coordinates
        delta[delta > NN_max_dist] = -L + delta[delta > NN_max_dist]
        delta[delta < -NN_max_dist] = L + delta[delta < -NN_max_dist]
       
        delta_tot[i] = delta
        r += delta**2   
        r = np.sqrt(r) 
        
    return (delta_tot, r)

def pot_and_force(delta_tot, r, dim, N):
    # Potentials and Forces
    F = np.zeros((dim,N,N),dtype=float)
    U = pot_LJ_dl(r)
    
    for i in range(dim):
        F[i][r != 0] = -grad_pot_LJ_dl(r[r != 0])*delta_tot[i][r != 0]/(r[r != 0])
    
    return(F, U)

In [203]:
# Plotting the potential for Argon
import matplotlib.pyplot as plt
k_b = 1.38e-23
argon = atom_props(3.405e-10, k_b*119.8)        

x_t = np.linspace(0.9,5,1000)
u_t = pot_LJ_DL(x_t);

plt.plot(x_t,u_t)
plt.xlabel(r'Distance r/$\sigma$')
plt.ylabel(r'Energy u/$\epsilon$')

   

In [204]:
L = 10 # Box size
N = 3 # partciles
dim = 2 # Dimensions of the probem

(x, v) = particle_generator(L, N, dim)
(delta_tot, r) = particle_dist(L, N, x, dim)
(F, U) = pot_and_force(delta_tot, r, dim, N)




##### Steps to be taken

    * U
    * F
    * vn+1
    * xn+1     